### First, remove all images of babies (age <= 5).
Because my name_ages.csv file has been corrupted, I have to re-look through all the images and see what the age is.

In [4]:
import os
all_images_dest = '../data/processed/all_images'
all_images = os.listdir(all_images_dest)

# First, remove all images from UTKFaces if they have age <= 5.
utk_count = 0
for image in all_images:
    if '_' in image:
        age = int(image.split('_')[0])

        if age >= 0 and age <= 5:
            image_path = f"{all_images_dest}/{image}"
            if os.path.exists(image_path):
                os.remove(image_path)
            utk_count += 1

print(f"Removed {utk_count} images of babies from the UTKFace dataset.")

Removed 2166 images of babies from the UTKFace dataset.


In [15]:
# Check if now all UTK images have age > 5.
all_images = os.listdir(all_images_dest)

all_utk_images = [int(img.split('_')[0]) for img in all_images if '_' in img]
print(sorted(all_utk_images))

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 

In [28]:
# Now, for the APPA-real dataset.
appa_ages_dict = {}
appa_path = '../data/raw/appa-real-release'
branches = ['test', 'valid', 'train']

for branch in branches:
    with open(f"{appa_path}/gt_avg_{branch}.csv", 'r') as f:
        contents = f.read().split('\n')

        for line in contents[1:-1]:
            image = line.split(',')[0]
            age = line.split(',')[-1]

            appa_ages_dict[image] = int(age)

#print(appa_ages_dict)

appa_count = 0
for image in all_images:
    if '_' in image:
        continue

    age = appa_ages_dict[image]
    if age >= 0 and age <= 5:
        image_path = f"{all_images_dest}/{image}"
        if os.path.exists(image_path):
            os.remove(image_path)
        appa_count += 1

print(f"Removed {appa_count} images of babies from the appa_real dataset.")

Removed 417 images of babies from the appa_real dataset.


In [29]:
# Check if now all appa_real images have age > 5.
all_images = os.listdir(all_images_dest)

for image in all_images:
    if '_' in image:
        continue

    age = appa_ages_dict[image]
    if age >= 0 and age <= 5:
        print("Incorrect picture found. Something's wrong!")


### Now that all images of babies have been removed, compute labels and save all data into a csv so it can be used easier.

In [30]:
all_images = os.listdir(all_images_dest)
csv_path = '../data/processed/name_age.csv'

with open(csv_path, 'w') as f:
    f.write('name,age\n')
    for image in all_images:
        if '_' in image:
            age = int(image.split('_')[0])
            f.write(f"{image},{age}\n")
        else:
            age = int(appa_ages_dict[image])
            f.write(f"{image},{age}\n")


In [32]:
# Check if everything is good.
with open(csv_path, 'r') as f:
    lines = f.readlines()
    assert (len(all_images) + 1) == len(lines)
    print("All ages have been saved successfully!")

All ages have been saved successfully!


### Now create a pandas DataFrame and cache it with pickle.

In [34]:
import pandas as pd
import pickle
pickle_file_path = './dataframe.pkl'
df = pd.read_csv(csv_path)

with open(pickle_file_path, 'wb') as f:
    pickle.dump(df, f)